In [ ]:
from math import sin, cos, sqrt, atan2, radians
import geopandas
import contextily as ctx
import matplotlib.pyplot as plt
import mercantile as mt
from tqdm import tqdm_notebook
import requests
import shutil
import os
from skimage import io
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries, find_boundaries
import numpy as np
from PIL import Image
import descartes
import shapely
import scipy
from scipy.ndimage import distance_transform_edt as distance_transform
import time
import geopy.distance

#The api key for bing maps:
BINGKEY = '######'

In [ ]:
#gt = geopandas.read_file('original.geojson')
#gt_3857 = gt.to_crs(epsg=3857) # Transform to Web Mercator
#gt_4326 = gt.to_crs(epsg=4326) # Transform to Lat/Long
tr = geopandas.read_file('displaced.geojson')
#tr_3857 = tr.to_crs(epsg=3857) # Transform to Web Mercator
tr_4326 = tr.to_crs(epsg=4326) # Transform to Lat/Long

In [ ]:
def plot_im(im, name):
    fig, ax = plt.subplots(1,1,figsize=(15,15))
    ax.imshow(im)
    ax.set_axis_off()
    plt.tight_layout()
    plt.savefig(name + '.png', dpi=200)

def plot2(im1, im2):
    fig, axs = plt.subplots(1,2, figsize=(13,7))
    axs[0].imshow(im1)
    axs[1].imshow(im2)

def add_basemap(ax, zoom, url='http://tile.stamen.com/terrain/tileZ/tileX/tileY.png'):
    xmin, xmax, ymin, ymax = ax.axis()
    basemap, extent = ctx.bounds2img(xmin, ymin, xmax, ymax, zoom=zoom, url=url)
    ax.imshow(basemap, extent=extent, interpolation='bilinear')
    # restore original x/y limits
    ax.axis((xmin, xmax, ymin, ymax))
    
def plot_mercator(df, zoom=17, margin=1):
    assert(df.crs['init'] == 'epsg:3857')
    ax = df.plot(figsize=(16, 16), alpha=0.5, edgecolor='k', color='b')
    ax.margins(margin)  
    add_basemap(ax, zoom=zoom, url=ctx.sources.OSM_A)
    ax.set_axis_off()
    
def plot_offset(idx, zoom=17, margin=1):
    ax = gt_3857.loc[idx:idx].plot(figsize=(16,16), alpha=0., edgecolor='k', color='g')
    tr_3857.loc[idx:idx].plot(figsize=(16,16), alpha=0., edgecolor='k', color='r', ax=ax)
    ax.margins(margin)  
    add_basemap(ax, zoom=zoom, url=ctx.sources.OSM_A)
    ax.set_axis_off()
    plt.tight_layout()
    plt.savefig('offset.png', dpi=200)
    
def plot_hm(im, hm):
    fig, ax = plt.subplots(1,1, figsize=(10,10))
    y,x,c = im.shape
    ax.imshow(hm, alpha=1, extent=(0,x,0,y))
    ax.imshow(im, alpha=0.5)
    ax.set_axis_off()
    plt.tight_layout()
    
def differences(gt, tr):
    diffs = []
    for i in tqdm_notebook(range(len(gt))):
        _gt = gt.loc[i].geometry.centroid
        _tr = tr.loc[i].geometry.centroid
        diffs.append(sqrt((_gt.x - _tr.x)**2 + (_gt.y - _tr.y)**2))
    return diffs

def overlaps(gt, tr):
    overlaps = 0
    for i in tqdm_notebook(range(len(gt))):
        if gt.loc[i].geometry.overlaps(tr.loc[i].geometry):
            overlaps += 1
    return overlaps / len(gt)

def save_footprints(df):
    for idx,sample in df.iterrows():
        #sample = df.loc[idx]
        savepath = '../footprints/{}.png'.format(sample.OBJECTID)
        fig = plt.figure(figsize=(7,7))
        ax = fig.add_subplot(111)
        patch = descartes.PolygonPatch(sample.geometry, fc='#000000')
        ax.add_patch(patch)
        plt.autoscale(True)
        ax.set_aspect('equal', adjustable='box')
        ax.set_axis_off()
        plt.savefig(savepath, bbox_inches='tight')
        plt.close(fig)

def gather_bing(df, bingkey, zoom=19, tilewidth=256):
    assert(df.crs['init'] == 'epsg:4326')
    exh = 50 # The extra height of the tiles to get rid of the watermark
    link = 'https://dev.virtualearth.net/REST/v1/Imagery/Map/Aerial/{lat},{lng}/{zoom}?mapSize={sizex},{sizey}&zoomLevel={zoom}&key={key}'
    lngmin, latmin, lngmax, latmax = df.total_bounds
    tiles = list(mt.tiles(lngmin, latmin, lngmax, latmax, zoom))
    for tile in tqdm_notebook(tiles):
        bd = mt.xy_bounds(tile)
        xmid = (bd.right + bd.left)/2
        ymid = (bd.top + bd.bottom)/2
        center = mt.lnglat(xmid, ymid)
        uri = link.format(lat=center.lat, lng=center.lng, zoom=zoom, sizex=tilewidth, sizey=tilewidth + 2*exh, key=bingkey)
        download_tile(uri, tilepath(tile))

def download_tile(url, savepath, overwrite=False):
    if not overwrite and os.path.isfile(savepath):
        return
    response = requests.get(url, stream=True)
    with open(savepath, 'wb') as out_file:
        shutil.copyfileobj(response.raw, out_file)
    del response

def tilepath(tile):
    outdir='data/'
    filename = 'x_{x}_y_{y}.jpg'
    return outdir + filename.format(x=tile.x,y=tile.y)

def crop_image(image ,tol=0):
    coords = np.argwhere(image)

    # Bounding box of non-black pixels.
    x0, y0 = coords.min(axis=0)
    x1, y1 = coords.max(axis=0) + 1   # slices are exclusive at the top

    # Get the contents of the bounding box.
    return image[x0:x1, y0:y1]

def pad(im, w):
    new_im = np.zeros((w,w))
    nw = int((w - im.shape[0]) / 2)
    nh = int((w - im.shape[1]) / 2)
    new_im[nw:nw + im.shape[0], nh:nh + im.shape[1]] = im
    return new_im

In [ ]:
def load_tile(tile):
    img = io.imread(tilepath(tile))
    return img[50:-50,:]

def load_footprint(idx):
    footprint = io.imread('footprints/{}.png'.format(idx+1))
    footprint = footprint[:,:,:-1]
    return footprint

def load_building(idx, scale=2.5):
    building = tr_4326.loc[idx]
    footprint = load_footprint(idx)
    scaled_building = shapely.affinity.scale(building.geometry, scale, scale)
    tiles = list(mt.tiles(*scaled_building.bounds, 19))
    images = [load_tile(tile) for tile in tiles]
    satim = stitch_tiles_images(tiles, images)
    fig, ax = plt.subplots(1,2, figsize=(14,10))
    ax[0].imshow(satim)
    ax[1].imshow(footprint)
    return building, footprint, satim
    
def stitch_tiles_images(tiles, images):
    ximg = images[0]
    lastx = tiles[0].x
    lasty = 0
    for tile, image in zip(tiles[1:], images[1:]):
        if tile.x > lastx:
            ximg = np.pad(ximg, [(0,0),(0,256),(0,0)], 'constant',  constant_values=0)
            lastx = tile.x
            lasty = 0
        if lastx == tiles[0].x:
            ximg = np.concatenate((ximg, image), axis=0)
        else:
            ximg[lasty:lasty+256,-256:,:] = image
        lasty += 256
    return ximg

def map_scale_prec(latitude, zoom):
    return 156543.03 * cos(latitude) / (2**zoom)

In [ ]:
def method_one(IDX):
    building, footprint, satellite_image = load_building(IDX)

    width, height = satellite_image.shape[:-1]
    lon1,lat1,lon2,lat2 = building.geometry.bounds
    building_diag_px = 0.9 * geopy.distance.distance((lon1,lat1),(lon2,lat2)).m / map_scale_prec(lat1, 19)

    footprint = crop_image(footprint[:,:,0] == 0)
    footprint_diag_px = sqrt(2*footprint.shape[0]**2)
    footprint = scipy.ndimage.zoom(footprint.astype('float'), building_diag_px / footprint_diag_px)
    footprint = (footprint > 0.5).astype('uint8') * 255

    # First SLIC
    first_order_segments = slic(satellite_image, n_segments=250, compactness=10, sigma=1)
    NSEG_1 = first_order_segments.max()

    # Mean Filling
    mean_image = np.zeros(satellite_image.shape).astype('uint8')
    for i in range(NSEG_1):
        binseg = first_order_segments == i
        mean_image[binseg] = satellite_image[binseg].mean(axis=0).astype('uint8')

    # Second order SLIC
    second_order_segments = slic(mean_image, n_segments=60, compactness=15, sigma=3)

    NSEG2 = second_order_segments.max()

    minerror = 1000000000000
    minidx = 0
    errs = []
    errims = []
    dists = []
    for i in tqdm_notebook(range(NSEG2)):
        segment = second_order_segments == i
        if segment.sum() == 0:
            continue
        segment_center = scipy.ndimage.measurements.center_of_mass(segment)
        distance_pixel = sqrt((width/2 - segment_center[0])**2 + (height/2 - segment_center[1])**2)

        cropped_segment = crop_image(segment).astype('uint8') * 255
        max_w = max(max(cropped_segment.shape), max(footprint.shape))
        cropped_footprint = np.array(pad(footprint, max_w)) == 255
        footprint_distance = 5 * distance_transform(1 - cropped_footprint) + distance_transform(cropped_footprint)
        cropped_segment = np.array(pad(cropped_segment, max_w)) == 255

        errim = (cropped_footprint != cropped_segment) * footprint_distance

        errims.append(errim)
        dists.append(distance_pixel)
        err = np.sum(errim) + distance_pixel * 100
        errs.append(err)
        if err < minerror:
            minerror = err
            minidx = i

    winning_segment = second_order_segments == minidx
    plt.figure(figsize=(10,10))
    plot_im(mark_boundaries(satellite_image, first_order_segments), 'first_order_segments')
    plot_im(mean_image, 'mean_image')
    plot_im(mark_boundaries(mean_image, second_order_segments), 'second_order_segments')
    plot_hm(satellite_image, np.flipud(winning_segment))

In [ ]:
def method_two(IDX):
    building, footprint, satellite_image = load_building(IDX)

    width, height = satellite_image.shape[:-1]
    lon1,lat1,lon2,lat2 = building.geometry.bounds
    building_diag_px = 0.9 * geopy.distance.distance((lon1,lat1),(lon2,lat2)).m / map_scale_prec(lat1, 19)

    footprint = crop_image(footprint[:,:,0] == 0)
    footprint_diag_px = sqrt(2*footprint.shape[0]**2)
    footprint = scipy.ndimage.zoom(footprint.astype('float'), building_diag_px / footprint_diag_px)
    footprint = (footprint > 0.5).astype('uint8') * 255
    footprint_bounds = find_boundaries(footprint)

    # First SLIC
    first_order_segments = slic(satellite_image, n_segments=250, compactness=10, sigma=1)
    distance_segbounds = distance_transform(1 - find_boundaries(first_order_segments))
    stride = 10

    errs = np.inf * np.ones(distance_segbounds.shape) 
    ps = footprint_bounds.shape
    for x1 in range(0, distance_segbounds.shape[0] - ps[0], stride):
        for x2 in range(0, distance_segbounds.shape[1] - ps[1], stride):
            m_x = int((x1 + x1 + ps[0])/2)
            m_y = int((x2 + x2 + ps[1])/2)
            errs[m_x][m_y] = np.sum(distance_segbounds[x1:x1 + ps[0], x2:x2 + ps[1]] * footprint_bounds)


    idx = np.unravel_index(errs.argmin(), errs.shape)
    hm = np.zeros(satellite_image.shape[:-1])
    hm[idx[0] - int(ps[0]/2) : idx[0] - int(ps[0]/2) + ps[0], idx[1] - int(ps[1]/2) : idx[1] - int(ps[1]/2) + ps[1]] = footprint

    plot_im(mark_boundaries(satellite_image, first_order_segments), 'first_order_segments')
    plot_hm(satellite_image, np.flipud(hm))

In [ ]:
method_one(12)

In [ ]:
method_two(12)